<img src="https://reliance.rohub.org/static/media/Reliance-logo.f1fd2415.png" width="250" align="left">
<img src="https://git.man.poznan.pl/stash/projects/ROHUB/repos/rohub-api/browse/rohub_logo.png?raw=" width="200" align="right">

# Creation of a Research Object with the Reliance (rohub) library aggregating resources from local and from the EGI notebook

Steps 
- Create a RO
- Add metadata - collateral information
- Add resources as internal
- Add resources as external by generating sharable links in the datahub

----

In [ ]:
pip install rohub

In [ ]:
import os
from rohub import rohub, settings
import pathlib
from zipfile import ZipFile
import json
import requests
import datetime
import shutil

## Sign in

In [ ]:
import getpass
rohub_user = "ADD USERNAME"

In [ ]:
rohub_pwd = getpass.getpass(prompt="Enter ROHub password: ")

In [ ]:
rohub.login(username=rohub_user, password=rohub_pwd)

## Check my last ROs

In [ ]:
myros=rohub.list_my_ros()
myros[0:10]

## Create a new RO

In [ ]:
# RO TYPE
#'Basic Research Object', 'Process-centric Research Object', 'Workflow-centric Research Object', 'Bibliography-centric Research Object', 
# 'Executable Research Object', 'Service-centric Research Object', 'Software-centric Research Object', 'Data-centric Research Object'

ro_title="ADD TITLE"
ro_research_areas=["ADD AREAS"]
ro_type='ADD TYPE'

ro = rohub.ros_create(title=ro_title, research_areas=ro_research_areas, ros_type=ro_type,use_template=True)

## ...or Load a existing RO

In [ ]:
ro_id=myros.loc[0,"identifier"]
ro=rohub.ros_load(identifier=ro_id)

### Show metadata

In [ ]:
ro.show_metadata()

### Add description to the RO

In [ ]:
ro.description="ADD DESCRIPTION"

ro.update()

### Add authors/contributors/publishers/copyrightHolders

In [ ]:
usernames=rohub.users_find(search="NAME1")
usernames

In [ ]:
user_id_N1=usernames.loc[0,"username"]

In [ ]:
usernames=rohub.users_find(search="NAME2")
usernames

In [ ]:
user_id_N2=usernames.loc[2,"username"]

In [ ]:
ro.set_authors(agents=[user_id_N1,user_id_N1])

### Add Funder

In [ ]:
funded_by = {
"grant_id": "ID",
"grant_Name": "NAME",
"grant_title": "TITLE",
"funder_name": "FUNDER",
"funder_doi": "DOI",
}

if funded_by:
    ro.add_funding(grant_identifier=funded_by["grant_id"], grant_name=funded_by["grant_Name"],
                   funder_name=funded_by["funder_name"], grant_title=funded_by["grant_title"],
                   funder_doi=funded_by["funder_doi"])

### See licenses

In [ ]:
licenses = rohub.list_available_licenses()
licenses

## Add license

In [ ]:
license = 'LICENSE'

ro.set_license(license_id=license) 

# Add resources to the RO

## Define folders

In [ ]:
WORKDIR_FOLDER = "ADD PATH"
print("WORKDIR FOLDER: ", WORKDIR_FOLDER)

In [ ]:
INPUT_DATA_DIR  = "ADD PATH"
OUTPUT_DATA_DIR = "ADD PATH"
TOOL_DATA_DIR   = "ADD PATH"

## Add folders

In [ ]:
ro.add_folders(name="FOLDER1")
myfolders = ro.list_folders()

## Add Sketch - Internal resource

In [ ]:
resi_res_type = "Sketch"
resi_file_path = "ADD PATH"
resi_title="TITLE"
resi_folder="FOLDER"
resi_description=resi_title

ro.add_internal_resource(res_type=resi_res_type,
                         file_path=resi_file_path,
                         title=resi_title,
                         description=resi_description,
                         folder=resi_folder)

## or from external resource

In [ ]:
resi_res_type = "Sketch"
resi_file_url = "URL"
resi_title="TITLE"
resi_folder="FOLDER"
resi_description=resi_title

ro.add_external_resource(res_type=resi_res_type,
                         input_url=resi_file_url,
                         title=resi_title,
                         description = resi_description,
                         folder=resi_folder)

## Add bibliographic resource

In [ ]:
rese_res_type="Bibliographic Resource"
rese_file_url="URL"
rese_title="TITLE"
rese_descr= "DESCRIPTION"
rese_folder="FOLDER"

ro.add_external_resource(res_type=rese_res_type,
                         input_url=rese_file_url,
                         title=rese_title,
                         description = rese_descr,
                         folder=rese_folder)

In [ ]:
ro.list_resources()

## Add Python Script or Jupyter Notebook as an internal resource

In [ ]:
import pathlib
resi_file_path="ADD PATH"

In [ ]:
if pathlib.Path(resi_file_path).exists():
    resi_res_type="Python Script"
    resi_title="TITLE"
    resi_description= resi_title
    resi_folder="FOLDER"
    ro.add_internal_resource(res_type=resi_res_type,
                             file_path=resi_file_path, 
                             title=resi_title, 
                             description=resi_description,
                            folder=resi_folder)

## or as an external executable resource

###  Get shared link from EGI datahub

In [ ]:
def egi_datahub_init():
    import requests, json, datetime
    
    ONEZONE_ENDPOINT="https://datahub.egi.eu/api/v3/onezone/"
    ONEPROVIDER_ENDPOINT="https://cesnet-oneprovider-01.datahub.egi.eu/api/v3/oneprovider/"
    
    with open("/var/run/secrets/egi.eu/access_token") as f:
        EGI_TOKEN = f.read()
        try:
            # get DATAHUB token
            headers = {
                'X-Auth-Token': f"egi:" + EGI_TOKEN,
                'Content-type': 'application/json',
            }
            # get current timestamp
            ts = datetime.datetime.now().timestamp()
            data = json.dumps({ 
                'name': 'REST and CDMI access token ' + str(ts), 
                'type': { 
                    'accessToken': {} 
                }, 
                'caveats': [ { 
                    'type': 'interface', 
                    'interface': 'rest' 
                }] 
            })
            response = requests.post(ONEZONE_ENDPOINT+'user/tokens/named', headers=headers, data=data)
            print(json.dumps(response.json(), indent=2))
            DATAHUB_TOKEN=response.json()['token']              
            return DATAHUB_TOKEN
        except:
            print("EGI Datahub Authentication problem: check your credentials")

In [ ]:
def egi_datahub_getlink(datahub_token, filename):
    import requests, json, os
    OIDC_ENDPOINT="https://aai.egi.eu/oidc/"
    ONEZONE_ENDPOINT="https://datahub.egi.eu/api/v3/onezone/"
    ONEPROVIDER_ENDPOINT="https://cesnet-oneprovider-01.datahub.egi.eu/api/v3/oneprovider/"
    
    bname = os.path.basename(filename)
    hname = filename.split('datahub/')[1]
    ## get file id
    headers = { 
        'X-Auth-Token': datahub_token 
    }
    response = requests.post(ONEPROVIDER_ENDPOINT+'lookup-file-id/'+hname, headers=headers)
    dh_fileid = response.json()['fileId']
    ## get shared link 
    headers = { 'X-Auth-Token': datahub_token, 'Content-Type': 'application/json',}
    data = json.dumps({ 
        'name': bname,
        'fileId': dh_fileid
    })
    response = requests.post(ONEPROVIDER_ENDPOINT+'shares', headers=headers, data=data)
    shareIdGenerated=response.json()['shareId']
    # get shared link details
    headers = {
        'X-Auth-Token': datahub_token
    }
    response = requests.get(ONEPROVIDER_ENDPOINT+'shares/'+shareIdGenerated, headers=headers)
    publicURL=response.json()['publicUrl'] #publicURL is nice link but to webpage where you can download file, not the actual file
    contentURL= ONEZONE_ENDPOINT+'shares/data/'+response.json()['rootFileId']+'/content' #use this link to aggregate Jupyter Notebooks
    return (publicURL, contentURL)

### EGI DataHub initialization

In [ ]:
DATAHUB_TOKEN = egi_datahub_init()

In [ ]:
import os
WORKDIR_FOLDER = "PATH"
INPUT_DATA_DIR  = "PATH"

myfilename = 'FILE'
shared_res_path = os.path.join(INPUT_DATA_DIR, myfilename)
print(shared_res_path)
links = egi_datahub_getlink(DATAHUB_TOKEN, shared_res_path)
# we can aggregate either the publicURL or the contentURL as external resource in the research object
# the publicURL redirects to the EGI DataHub Webpage of the resource, where you can download the resource
# the contentURL redirects to the file contents directly. 
# NOTE: YOU NEED TO USE contentURL to aggregate JupyterNoteboks to be able to execute them from ROHub
res_file_url_public = links[0] 
print (res_file_url_public)
res_file_url_content = links[1] 
print (res_file_url_content)

### Add the resource to our new RO

In [ ]:
res_type = "Jupyter Notebook"
res_title = "TITLE"
res_description = "DESCRIPTION"
res_folder='FOLDER'
ro.add_external_resource(res_type=res_type,
                         input_url=res_file_url_content,
                         title=res_title, 
                         description=res_description, 
                         folder=res_folder)